# Simple binary classification problem utilizing convolutional neural networks 

## Import libraries 

In [35]:
# Import libraries. 
from __future__ import print_function
import os
import sys
#os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu0,floatX=float32,optimizer=fast_compile'
#os.environ['KERAS_BACKEND'] = 'theano'
# """
# os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu3,floatX=float32,optimizer=fast_compile'
# os.environ['KERAS_BACKEND'] = 'theano'

# In case you want to select a graphic card (i the above code i set the 3rd graphic card.) 
# """
#from keras.preprocessing.image import ImageDataGenerator

from keras.api.models import Sequential
from keras.api.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.api.layers import Convolution2D, MaxPooling2D
from keras.api.optimizers import SGD
from keras.api.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
import numpy as np
import keras 
import keras.api.backend as K
from keras.api.callbacks import LearningRateScheduler
import math
from keras import callbacks
import glob
from PIL import Image
from keras.src.utils import plot_model
import h5py


## Basic functions 

In [58]:
# It is good to know the pid of the running code in case you need to stop  or monitor. 
print (os.getpid())
file_open = lambda x,y: glob.glob(os.path.join(x,y))

# learning rate schedule. It is helpful when the learning rate can be dynamically set up. We will be using the callback functionality that keras provides. 
def step_decay(epoch):
  initial_lrate =0.01
  drop = 0.3
  epochs_drop = 30.0
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  print (lrate)
  return lrate

# The following function will be used to give a number of the parameters in our model. Useful when we need to get an estimate of what size of dataset we have to use.  
def size(model): 
  return sum([np.prod(K.get_value(w).shape) for w in model.trainable_weights])

def createmodel(img_channels,img_rows,img_cols):
  # This is a Sequential model. Graph models can be used in order to create more complex networks. 
  # Teaching Points:
  # 1. Here we utilize the adam optimization algorithm. In order to use the SGD algorithm one could replace the {adam=keras.optimizers.Adadelta(lr=0)} line with  {sgd = SGD(lr=0.0, momentum=0.9, decay=0.0, nesterov=False)} make sure you import the correct optimizer from keras. 
  # 2. This is a binary classification problem so make sure that the correct activation loss function combination is used. For such a problem the sigmoid activation function with the binary cross entropy loss is a good option
  # 3. Since this is a binary problem use   model.add(Dense(1)) NOT 2...
  # 4. For multi class model this code can be easily modified by selecting the softmax as activation function and the categorical cross entropy as loss 

  model = Sequential()
  #model.add(Convolution2D(16, 3, 3, padding='same',input_shape=(1, img_rows, img_cols)))
  model.add(Convolution2D(16, 3, 3, padding='same',input_shape=(img_cols, img_rows, 1)))
  model.add(Activation('relu'))
  model.add(Convolution2D(16, 5, 5, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
  model.add(Convolution2D(32, 3, 3, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Convolution2D(64, 5, 5, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Convolution2D(64, 3, 3, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
  model.add(Convolution2D(128, 3, 3, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
  model.add(Flatten())
  model.add(Dense(128, kernel_initializer='he_normal'))
  model.add(Activation('relu'))
  model.add(Dropout(0.5)) 
  model.add(Dense(32, kernel_initializer='he_normal'))
  model.add(Activation('relu'))
  model.add(Dropout(0.5)) 
  model.add(Dense(1))
  # model.add(Activation('relu'))
  model.add(Activation('sigmoid'))
  # learning schedule callback
  adam=keras.optimizers.Adadelta(learning_rate=0.0)
  lrate = LearningRateScheduler(step_decay)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
  
  # requires graphviz downloaded to work
  #plot_model(model, to_file='model.png')
  print(model.summary())
  return model

def shuffle(X, y):
  perm = np.random.permutation(len(y))
  X = X[perm]
  y = y[perm]
  print (np.shape(X))
  return X, y


def read_data(image):
  "opens image and converts it to a m*n matrix" 
  image = Image.open(image)
  image = image.getdata()
  # image = list(image.getdata())
  # image = map(list,image)
  image = np.array(image)
  return image.reshape(-1)

def createTrainTestValset(image_dir1, image_dir2):
  Class1_images = file_open(image_dir1,"*.jpg")
  Class2_images = file_open(image_dir2,"*.jpg")
  Class1_set = []
  Class2_set = []
  # Read all the files, and create numpy arrays. 
  Class1_set = [read_data(image) for image in Class1_images]
  Class2_set = [read_data(image) for image in Class2_images]
  Class1_set = np.array(Class1_set) #This is where the Memory Error occurs
  Class2_set = np.array(Class2_set)
  X=np.vstack((Class1_set, Class2_set))
  X=X.astype(float)/255
  # print (np.shape(X))
  yclass1=np.zeros((np.shape(Class1_set)[0]))
  yclass2=np.ones((np.shape(Class2_set)[0]))
  # print (np.shape(yclass1))
  y=np.concatenate((yclass1, yclass2))
  # print (np.shape(y)) 
  X,y=shuffle(X, y)
  print (np.shape(X)) 
  print (np.max(X))
  print (np.shape(y)) 
  X_train, X_val,y_train, y_val= train_test_split(X,y, test_size=0.2, random_state=42)
  return X_train,y_train, X_val, y_val 

  # Read the images; and split them in three different sets. 
def trainandpredict(Scan=32 ,img_channels=1,batch_size=64,nb_epoch=5,data_augmentation=False):
  img_rows=Scan
  img_cols=Scan
  CurrentDir= os.getcwd()
  image_dir1=os.path.abspath(os.path.join(os.path.abspath(os.path.join(CurrentDir, os.pardir)), "Data","negative_images"))
#   print(image_dir1)
  image_dir2=os.path.abspath(os.path.join(os.path.abspath(os.path.join(CurrentDir, os.pardir)), "Data","positive_images"))
  modeleval=createmodel(img_channels,img_rows,img_cols)
  X_train,y_train, X_val, y_val = createTrainTestValset(image_dir1, image_dir2)
  # X_train =X_train.reshape(
  #   -1,  # number of samples, -1 makes it so that this number is determined automatically
  #   1,   # 1 color channel, since images are only black and white
  #   Scan,  # first image dimension (vertical)
  #   Scan,  # second image dimension (horizontal)
  # )
  # X_val =X_val.reshape(
  #   -1,  # number of samples, -1 makes it so that this number is determined automatically
  #   1,   # 1 color channel, since images are only black and white
  #   Scan,  # first image dimension (vertical)
  #   Scan,  # second image dimension (horizontal)
  # )

  X_train =X_train.reshape(
    -1,  # number of samples, -1 makes it so that this number is determined automatically
    Scan,  # first image dimension (vertical)
    Scan,  # second image dimension (horizontal)
    1,   # 1 color channel, since images are only black and white
  )
  X_val =X_val.reshape(
    -1,  # number of samples, -1 makes it so that this number is determined automatically
    Scan,  # first image dimension (vertical)
    Scan,  # second image dimension (horizontal)
    1,   # 1 color channel, since images are only black and white
  )


  # Callbacks
  best_model = ModelCheckpoint('Final.weights.h5', verbose=1, monitor='val_loss',save_best_only=True,save_weights_only=True)
  lrate = LearningRateScheduler(step_decay)

  # Data augmentation is always a good choice
  if not data_augmentation:
    print('Not using data augmentation.')
    modeleval.load_weights('Final.weights.h5')
    modeleval.fit(X_train, y_train,batch_size=batch_size,epochs=nb_epoch,validation_split=0.1,callbacks=[best_model,lrate],shuffle=True)
  else:
    print('Using real-time data augmentation.')
    print ("pending")
    sys.exit()
    # this will do preprocessing and realtime data augmentation
    # datagen = ImageDataGenerator(
    #   featurewise_center=False,  # set input mean to 0 over the dataset
    #   samplewise_center=False,  # set each sample mean to 0
    #   featurewise_std_normalization=False,  # divide inputs by std of the dataset
    #   samplewise_std_normalization=False,  # divide each input by its std
    #   zca_whitening=False,  # apply ZCA whitening
    #   rotation_range=3,  # randomly rotate images in the range (degrees, 0 to 180)
    #   width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    #   height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    #   horizontal_flip=True,  # randomly flip images
    #   vertical_flip=False)  # randomly flip images
    # modeleval.fit(X_train, y_train,batch_size=batch_size,nb_epoch=nb_epoch,validation_data=(X_train1, y_train1),callbacks=[best_model,lrate],shuffle=True)


  # Some evaluation Just the basic stuff... 
  print (dir(modeleval))
  Y_cv_pred = modeleval.predict(X_val, batch_size = 32)
  roc =roc_auc_score(y_val, Y_cv_pred)
  print("ROC:", roc)
  print (Y_cv_pred)
  Y_cv_pred[Y_cv_pred>=.5]=1
  Y_cv_pred[Y_cv_pred<.5]=0
  target_names=[] 
  # print ("The f1-score gives you the harmonic mean of precision and recall. The scores corresponding to every class will tell you the accuracy of the classifier in classifying the data points in that particular class compared to all other classes.The support is the number of samples of the true response that lie in that class.")
  target_names = ['class 0', 'class 1']
  print(classification_report(y_val, Y_cv_pred, target_names=target_names,digits=4))





18760


## Run Program 

In [59]:
if __name__ == '__main__':
    trainandpredict()

c:\Data\VSCodeProjects\MachineLearning\ProjectCode\virtualenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_162 (Conv2D)             │ (None, 11, 11, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_240 (Activation)     │ (None, 11, 11, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_163 (Conv2D)             │ (None, 3, 3, 16)       │         6,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_135         │ (None, 3, 3, 16)       │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_241 (Activation)     │ (None, 3, 3, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_81 (MaxPooling2D) │ (None, 2, 2, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_164 (Conv2D)             │ (None, 1, 1, 32)       │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_136         │ (None, 1, 1, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_242 (Activation)     │ (None, 1, 1, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_165 (Conv2D)             │ (None, 1, 1, 64)       │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_137         │ (None, 1, 1, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_243 (Activation)     │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_166 (Conv2D)             │ (None, 1, 1, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_138         │ (None, 1, 1, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_244 (Activation)     │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_82 (MaxPooling2D) │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_167 (Conv2D)             │ (None, 1, 1, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_139         │ (None, 1, 1, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_245 (Activation)     │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_83 (MaxPooling2D) │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_27 (Flatten)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_79 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 195,153 (762.32 KB)

 Trainable params: 194,545 (759.94 KB)

 Non-trainable params: 608 (2.38 KB)

None
(8710, 1024)
(8710, 1024)
1.0
(8710,)
Not using data augmentation.
0.01
Epoch 1/5


c:\Data\VSCodeProjects\MachineLearning\ProjectCode\virtualenv\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adadelta', because it has 2 variables whereas the saved optimizer has 58 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


96/98 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6522 - loss: 0.6573
Epoch 1: val_loss improved from inf to 0.57256, saving model to Final.weights.h5
98/98 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.6524 - loss: 0.6571 - val_accuracy: 0.7288 - val_loss: 0.5726 - learning_rate: 0.0100
0.01
Epoch 2/5
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6551 - loss: 0.6447
Epoch 2: val_loss improved from 0.57256 to 0.51951, saving model to Final.weights.h5
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6552 - loss: 0.6445 - val_accuracy: 0.7461 - val_loss: 0.5195 - learning_rate: 0.0100
0.01
Epoch 3/5
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6799 - loss: 0.6136
Epoch 3: val_loss improved from 0.51951 to 0.48948, saving model to Final.weights.h5
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6798 - loss: 0.6136 - val_accuracy: 0.7590 - val_loss: 0.4895 - learning_rate: 0.0100
0.01
Epoch 4/5
97/98 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6862 - l